<a href="https://colab.research.google.com/github/Nithyarajoman/Master-Project-Repository/blob/main/ResNet18_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [ ]:
# Function to calculate return MAC from each layer

def mac_each_layer(l):
    o_shape, i_shape, ks, filters = ['', '', ''], ['', '', ''], [0, 0], [0, 0]
    macc = 0
    if ('Dense' in str(l)):
        print(l.input_shape)
        print(l.output_shape)

        i_shape = l.input.shape[1:4].as_list()[0]
        o_shape = l.output.shape[1:4].as_list()
        print(i_shape)
        print(o_shape)
        macc = (o_shape[0] * i_shape)

    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]
        macc = (filters * o_shape[0] * o_shape[1]) * (i_shape[2] * ks[0] * ks[1] )
    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = (o_shape[0] *o_shape[1]) * (i_shape[2] * ks[0] * ks[1] )

    return macc

# Function to pass layers to the calculating function and to get the macc of each layer

def MAC_summary(model):
  print('%25s | %6s' % ('Layer Name', 'MACs'))
  print('*' * 100)
  t_macc = 0
  macc_dict = {}
  prefix_name = model.name + "_"
  for l in model.layers:
    if (type(l) in [models.Sequential, keras.Model]):
      for ib_layer in l.layers:
        mac_obtained = mac_each_layer(ib_layer)
        macc_dict[prefix_name+l.name] = mac_obtained
        if('conv' in l.name):
          print('%25s(%5s) | %5.4f' % (
                  prefix_name+l.name,ib_layer.name, mac_obtained ))
        t_macc += mac_obtained

    else:
      mac_obtained = mac_each_layer(l)
      macc_dict[prefix_name+l.name] = mac_obtained
      if('conv' in l.name):
        print('%25s | %5.4f' % (
                prefix_name+l.name, mac_obtained ))

    t_macc += mac_obtained
  print('\nTotal MACs operations in the complete model: %10.8f\n' % (t_macc))
  return macc_dict

In [ ]:
# Obtains the output in bits, the memory requirements of each layer

def keras_model_output(model):
    print('%25s | %6s' % ('Layer Name', 'Output bits'))
    print('*' * 100)
    output_dict = {}
    default_dtype = tf.keras.backend.floatx()
    prefix_name = model.name + "_"
    for layer in model.layers:
        if isinstance(layer, (tf.keras.Model,models.Sequential)):
            keras_model_output(layer)
        else:
            no_of_neurons = tf.as_dtype(layer.dtype or default_dtype).size # dtype is float 32 and the size here is 4 bytes(32 bits)
            out_shape = layer.output_shape
            if isinstance(out_shape, list):
                for s in out_shape[0]:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes
            else:
                for s in out_shape:
                    if s is None:
                        continue
                    no_of_neurons *= s # Output obtained here is in 4 Bytes
            no_of_neurons *= 8 # Output obtained in bits
        output_dict[prefix_name + layer.name] = no_of_neurons
        print('%25s | %5.4f' % (
                  prefix_name+layer.name, no_of_neurons ))


    return output_dict

In [ ]:
# EFFECTIVE DATA COMMUNICATION PROBABILITY

# Function to print the effective data communicated
def data_summary(model,x_test,y_test):

  # Creating dictionary for storing the percent of data communicated eliminating the sparse data
  effective_probability = {}

  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"

    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)

    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)

  # Compiling the cloned model and take the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  # m2.fit(x_test, y_test, epochs=5)
  m2.evaluate(x_test[0:100], y_test[0:100])

  print('%25s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  print("*" * 150)
  for l in m2.metrics:
    # print('%40s | %5.4f' % (l.name, l.result().numpy()))
    if l.name!="loss" and l.name!="accuracy" :
      effective_probability[l.name] = l.result().numpy()
      print('%25s | %5.4f' % (
                  l.name, effective_probability[l.name] ))

  return effective_probability


In [ ]:
# ImageNet classification dataset for Resnet18 and ResNet50
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data() #Size is (, 32,32,3)

169001437/169001437 [==============================] - 2s 0us/step


In [ ]:
# Calculations for Resnet18 CNN model
!pip install image-classifiers
from tensorflow.keras.applications import imagenet_utils
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model_ResNet18 = ResNet18(include_top=False,
                        input_tensor=None,
                        input_shape=(
                            32, 32, 3),
                        pooling=None)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
MAC_summary(model_ResNet18)

               Layer Name |   MACs
****************************************************************************************************
              model_conv0 | 2408448.0000
 model_stage1_unit1_conv1 | 2359296.0000
 model_stage1_unit1_conv2 | 2359296.0000
 model_stage1_unit2_conv1 | 2359296.0000
 model_stage1_unit2_conv2 | 2359296.0000
 model_stage2_unit1_conv1 | 1179648.0000
 model_stage2_unit1_conv2 | 2359296.0000
 model_stage2_unit2_conv1 | 2359296.0000
 model_stage2_unit2_conv2 | 2359296.0000
 model_stage3_unit1_conv1 | 1179648.0000
 model_stage3_unit1_conv2 | 2359296.0000
 model_stage3_unit2_conv1 | 2359296.0000
 model_stage3_unit2_conv2 | 2359296.0000
 model_stage4_unit1_conv1 | 1179648.0000
 model_stage4_unit1_conv2 | 2359296.0000
 model_stage4_unit2_conv1 | 2359296.0000
 model_stage4_unit2_conv2 | 2359296.0000

Total MACs operations in the complete model: 37273600.00000000



{'model_data': 0,
 'model_bn_data': 0,
 'model_zero_padding2d': 0,
 'model_conv0': 2408448,
 'model_bn0': 0,
 'model_relu0': 0,
 'model_zero_padding2d_1': 0,
 'model_pooling0': 0,
 'model_stage1_unit1_bn1': 0,
 'model_stage1_unit1_relu1': 0,
 'model_zero_padding2d_2': 0,
 'model_stage1_unit1_conv1': 2359296,
 'model_stage1_unit1_bn2': 0,
 'model_stage1_unit1_relu2': 0,
 'model_zero_padding2d_3': 0,
 'model_stage1_unit1_conv2': 2359296,
 'model_stage1_unit1_sc': 262144,
 'model_add': 0,
 'model_stage1_unit2_bn1': 0,
 'model_stage1_unit2_relu1': 0,
 'model_zero_padding2d_4': 0,
 'model_stage1_unit2_conv1': 2359296,
 'model_stage1_unit2_bn2': 0,
 'model_stage1_unit2_relu2': 0,
 'model_zero_padding2d_5': 0,
 'model_stage1_unit2_conv2': 2359296,
 'model_add_1': 0,
 'model_stage2_unit1_bn1': 0,
 'model_stage2_unit1_relu1': 0,
 'model_zero_padding2d_6': 0,
 'model_stage2_unit1_conv1': 1179648,
 'model_stage2_unit1_bn2': 0,
 'model_stage2_unit1_relu2': 0,
 'model_zero_padding2d_7': 0,
 'model_

In [ ]:
keras_model_output(model_ResNet18)

               Layer Name | Output bits
****************************************************************************************************
               model_data | 98304.0000
            model_bn_data | 98304.0000
     model_zero_padding2d | 138624.0000
              model_conv0 | 524288.0000
                model_bn0 | 524288.0000
              model_relu0 | 524288.0000
   model_zero_padding2d_1 | 663552.0000
           model_pooling0 | 131072.0000
   model_stage1_unit1_bn1 | 131072.0000
 model_stage1_unit1_relu1 | 131072.0000
   model_zero_padding2d_2 | 204800.0000
 model_stage1_unit1_conv1 | 131072.0000
   model_stage1_unit1_bn2 | 131072.0000
 model_stage1_unit1_relu2 | 131072.0000
   model_zero_padding2d_3 | 204800.0000
 model_stage1_unit1_conv2 | 131072.0000
    model_stage1_unit1_sc | 131072.0000
                model_add | 131072.0000
   model_stage1_unit2_bn1 | 131072.0000
 model_stage1_unit2_relu1 | 131072.0000
   model_zero_padding2d_4 | 204800.0000
 model_stage1_unit2_c

{'model_data': 98304,
 'model_bn_data': 98304,
 'model_zero_padding2d': 138624,
 'model_conv0': 524288,
 'model_bn0': 524288,
 'model_relu0': 524288,
 'model_zero_padding2d_1': 663552,
 'model_pooling0': 131072,
 'model_stage1_unit1_bn1': 131072,
 'model_stage1_unit1_relu1': 131072,
 'model_zero_padding2d_2': 204800,
 'model_stage1_unit1_conv1': 131072,
 'model_stage1_unit1_bn2': 131072,
 'model_stage1_unit1_relu2': 131072,
 'model_zero_padding2d_3': 204800,
 'model_stage1_unit1_conv2': 131072,
 'model_stage1_unit1_sc': 131072,
 'model_add': 131072,
 'model_stage1_unit2_bn1': 131072,
 'model_stage1_unit2_relu1': 131072,
 'model_zero_padding2d_4': 204800,
 'model_stage1_unit2_conv1': 131072,
 'model_stage1_unit2_bn2': 131072,
 'model_stage1_unit2_relu2': 131072,
 'model_zero_padding2d_5': 204800,
 'model_stage1_unit2_conv2': 131072,
 'model_add_1': 131072,
 'model_stage2_unit1_bn1': 131072,
 'model_stage2_unit1_relu1': 131072,
 'model_zero_padding2d_6': 204800,
 'model_stage2_unit1_conv

In [ ]:
data_summary(model_ResNet18,x_train, y_train)


4/4 [==============================] - 7s 57ms/step - loss: 15.8941 - accuracy: 0.0000e+00 - model_data: 0.9591 - model_bn_data: 0.9591 - model_zero_padding2d: 0.6801 - model_conv0: 0.9900 - model_bn0: 0.9900 - model_relu0: 0.4550 - model_zero_padding2d_1: 0.3595 - model_pooling0: 0.6593 - model_stage1_unit1_bn1: 0.6593 - model_stage1_unit1_relu1: 0.6593 - model_zero_padding2d_2: 0.4219 - model_stage1_unit1_conv1: 1.0000 - model_stage1_unit1_bn2: 1.0000 - model_stage1_unit1_relu2: 0.5278 - model_zero_padding2d_3: 0.3378 - model_stage1_unit1_conv2: 1.0000 - model_stage1_unit1_sc: 0.9980 - model_add: 1.0000 - model_stage1_unit2_bn1: 1.0000 - model_stage1_unit2_relu1: 0.4365 - model_zero_padding2d_4: 0.2794 - model_stage1_unit2_conv1: 1.0000 - model_stage1_unit2_bn2: 1.0000 - model_stage1_unit2_relu2: 0.4612 - model_zero_padding2d_5: 0.2951 - model_stage1_unit2_conv2: 1.0000 - model_add_1: 1.0000 - model_stage2_unit1_bn1: 1.0000 - model_stage2_unit1_relu1: 0.4412 - model_zero_padding2d_6:

{'model_data': 0.9591013590494791,
 'model_bn_data': 0.9591013590494791,
 'model_zero_padding2d': 0.6801383598799631,
 'model_conv0': 0.989959716796875,
 'model_bn0': 0.989959716796875,
 'model_relu0': 0.4549980163574219,
 'model_zero_padding2d_1': 0.3595046055169753,
 'model_pooling0': 0.6592597961425781,
 'model_stage1_unit1_bn1': 0.6592597961425781,
 'model_stage1_unit1_relu1': 0.6592597961425781,
 'model_zero_padding2d_2': 0.42192626953125,
 'model_stage1_unit1_conv1': 1.0,
 'model_stage1_unit1_bn2': 1.0,
 'model_stage1_unit1_relu2': 0.5278129577636719,
 'model_zero_padding2d_3': 0.33780029296875,
 'model_stage1_unit1_conv2': 1.0,
 'model_stage1_unit1_sc': 0.998046875,
 'model_add': 1.0,
 'model_stage1_unit2_bn1': 1.0,
 'model_stage1_unit2_relu1': 0.43653297424316406,
 'model_zero_padding2d_4': 0.27938110351562495,
 'model_stage1_unit2_conv1': 1.0,
 'model_stage1_unit2_bn2': 1.0,
 'model_stage1_unit2_relu2': 0.4611701965332031,
 'model_zero_padding2d_5': 0.29514892578124996,
 'mode